In [33]:
%pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.3 MB/s eta 0:00:00


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import resample

In [4]:
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("validation.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df.head()

,Age,Class,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,...,Arrival Delay in Minutes,Departure Delay in Minutes,Flight Distance,Gender_Female,Gender_Male,Type of Travel_Business travel,Type of Travel_Personal Travel,Customer Type_Loyal Customer,Customer Type_disloyal Customer,satisfaction
0,0.089744,1.0,0.8,0.8,0.2,0.8,1.0,0.8,1.00,1.0,...,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1
1,0.410256,1.0,1.0,1.0,1.0,1.0,0.8,0.8,0.75,1.0,...,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1
2,0.230769,1.0,0.4,1.0,0.4,0.6,0.2,0.4,0.00,0.2,...,0.333333,0.5,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0
3,0.217949,0.5,1.0,0.0,1.0,0.6,0.4,1.0,0.25,0.4,...,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1
4,0.512821,1.0,0.2,0.2,0.2,0.2,0.4,1.0,0.75,0.8,...,0.333333,0.5,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1


In [6]:
train_df.columns

Index(['Age', 'Class', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Arrival Delay in Minutes', 'Departure Delay in Minutes',
       'Flight Distance', 'Gender_Female', 'Gender_Male',
       'Type of Travel_Business travel', 'Type of Travel_Personal Travel',
       'Customer Type_Loyal Customer', 'Customer Type_disloyal Customer',
       'satisfaction'],
      dtype='object')

In [8]:
features = train_df.columns.drop('satisfaction')
target = 'satisfaction'

In [15]:
# Sample 5% of the training data
train_sample = resample(train_df, n_samples=int(0.05 * len(train_df)), random_state=42)

In [16]:
X_train = train_sample[features]
y_train = train_sample[target]
X_val = val_df[features]
y_val = val_df[target]
X_test = test_df[features]
y_test = test_df[target]

In [17]:
mlp = MLPClassifier(hidden_layer_sizes=(10,), random_state=42)

In [18]:
mlp.fit(X_train, y_train)
mlp_val_pred = mlp.predict(X_val)
mlp_accuracy = accuracy_score(y_val, mlp_val_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [19]:
mlp_accuracy

0.909955343393902

In [22]:
mlp = MLPClassifier(hidden_layer_sizes=(50,), random_state=42)

In [23]:
mlp.fit(X_train, y_train)
mlp_val_pred = mlp.predict(X_val)
mlp_accuracy = accuracy_score(y_val, mlp_val_pred)
mlp_accuracy

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.9222359100708346

In [24]:
X_train = train_df[features]
y_train = train_df[target]
X_val = val_df[features]
y_val = val_df[target]
X_test = test_df[features]
y_test = test_df[target]

In [25]:
mlp = MLPClassifier(hidden_layer_sizes=(10,), random_state=42)

In [26]:
mlp.fit(X_train, y_train)
mlp_val_pred = mlp.predict(X_val)
mlp_accuracy = accuracy_score(y_val, mlp_val_pred)
mlp_accuracy

0.9469510317215891

In [27]:
mlp = MLPClassifier(hidden_layer_sizes=(50,), random_state=42)

In [28]:
mlp.fit(X_train, y_train)
mlp_val_pred = mlp.predict(X_val)
mlp_accuracy = accuracy_score(y_val, mlp_val_pred)
mlp_accuracy

0.9534955343393902

## Fine Tuning

In [35]:
# Optuna optimization function
def objective(trial):
    # Suggest hyperparameters
    n_layers = trial.suggest_int('n_layers', 1, 3)  # Number of layers
    hidden_layer_sizes = tuple(trial.suggest_int(f'n_units_l{i}', 10, 100) for i in range(n_layers))
    learning_rate_init = trial.suggest_loguniform('learning_rate_init', 0.0001, 0.1)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    solver = trial.suggest_categorical('solver', ['adam', 'sgd'])
    alpha = trial.suggest_loguniform('alpha', 0.00001, 0.01)  # L2 regularization

    # Initialize MLP
    mlp = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        learning_rate_init=learning_rate_init,
        activation=activation,
        solver=solver,
        alpha=alpha,
        max_iter=200,
        random_state=42
    )

    # Train and evaluate
    mlp.fit(X_train, y_train)
    val_pred = mlp.predict(X_val)
    accuracy = accuracy_score(y_val, val_pred)

    return accuracy


In [36]:
# Create study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-07-18 23:22:10,170] A new study created in memory with name: no-name-28fdb23f-218b-4c14-b60d-7a0659262929
<ipython-input-35-e5adba4ff3e5>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate_init = trial.suggest_loguniform('learning_rate_init', 0.0001, 0.1)
<ipython-input-35-e5adba4ff3e5>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 0.00001, 0.01)  # L2 regularization
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[I 2024-07-18 

In [37]:
# Best hyperparameters
best_params = study.best_params
print(f'Best Hyperparameters: {best_params}')

Best Hyperparameters: {'n_layers': 2, 'n_units_l0': 67, 'n_units_l1': 20, 'learning_rate_init': 0.010047325174366043, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.0017360992714110025}


In [38]:
best_mlp = MLPClassifier(
    hidden_layer_sizes=tuple(best_params[f'n_units_l{i}'] for i in range(best_params['n_layers'])),
    learning_rate_init=best_params['learning_rate_init'],
    activation=best_params['activation'],
    solver=best_params['solver'],
    alpha=best_params['alpha'],
    max_iter=200,
    random_state=42
)

In [39]:
best_mlp.fit(X_train, y_train)

MLPClassifier(alpha=0.0017360992714110025, hidden_layer_sizes=(67, 20),
              learning_rate_init=0.010047325174366043, random_state=42)

In [45]:
val_pred = best_mlp.predict(X_val)
val_accuracy = accuracy_score(y_val, val_pred)
print(f'Validation Accuracy: {val_accuracy*100:.3f}')

Validation Accuracy: 95.881


In [46]:
test_pred = best_mlp.predict(X_test)
test_accuracy = accuracy_score(y_test, test_pred)
print(f'Test Accuracy: {test_accuracy*100:.3f}')

Test Accuracy: 96.139
